# Knowledge Graph for Portuguese Higher Education 

Group 2

* 53819 - Gonçalo Garcias
* 60898 - João Silva
* 64374 - Frederico Cerqueira
* 64853 - Joana Chuço

## Problem

Access to higher education in Portugal involves the analysis of a large volume of data, published annually by the *Direção-Geral do Ensino Superior* (DGES). This data is made available in tabular formats, such as spreadsheets, which hinder its automatic exploration, integration, and reuse. Furthermore, the lack of semantic representation limits the ability to perform complex queries, combine information from different sources, and obtain a broader understanding of the educational offerings available in the country. This limitation affects not only researchers and public entities, but also students and guardians seeking to make informed decisions.

Although the DGES portal provides a search engine for courses and institutions, the navigation is not intuitive and the process of retrieving detailed information can be time-consuming. The available search options are based on fixed combinations such as course/institution, district/institution, and institution/course, with results displayed as long lists that make it difficult to compare and filter by relevant criteria. One of the main limitations is the inability to filter directly by admission grade, which makes it harder to identify courses or institutions based on a candidate's academic performance.


## Knowledge Graph

![My Diagram](docs/Graph_image.jpg)

Structure of the RDF Knowledge Graph for Higher Education Institutions in Portugal

In [14]:
import pandas as pd
import os
from rdflib import Graph, Namespace, Literal, RDF, XSD

### Creating of the kg

A custom namespace, EDU (http://example.org/education#), was created to model concepts such as Institution, Course, Degree, ScientificArea, District, and County, along with their properties and objects.

In [15]:
EDU = Namespace("http://example.org/education#")

To ensure the reliable and semantically consistent construction of the knowledge graph, several utility functions were implemented. These are essential for data normalization, literal creation, and graph manipulation.

The `get_value` function ensures that each value retrieved from the spreadsheet is treated as a clean, stripped string, preventing inconsistencies due to extra spaces or mixed data types. 

In [16]:
def get_value(row, column):
    """
    Get the value from a row, ensuring it's a string and stripped of whitespace.
    """
    return str(row[column]).strip()

The `add_to_graph` function wraps the addition of RDF triples to the graph, promoting clarity and reusability throughout the code.

In [17]:
def add_to_graph(graph, subject, predicate, obj):
    """
    Add a triple to the graph.
    """
    graph.add((subject, predicate, obj))

Literals in RDF require explicit datatypes to enable accurate querying and reasoning. To support this, the functions `create_literal`, `create_str_literal`, `create_int_literal`, and `create_float_literal` convert *Python* values into RDF literals using XML Schema datatypes. String values are encoded as `xsd:string`, integers as `xsd:integer`, and floating-point numbers as `xsd:float`. Special care is taken to handle placeholders such as "---" in the original data by converting them to 0.0, ensuring that numerical values remain valid and comparable in queries.

In [18]:
def create_literal(value, datatype):
    """
    Create a literal with the specified datatype.
    """
    return Literal(value, datatype=datatype)


def create_str_literal(value):
    """
    Create a string literal. 
    """
    return create_literal(value, datatype=XSD.string)


def create_float_literal(value):
    """
    Create a float literal. If the value is "---", it is converted to 0.0.
    """
    if value == "---":
        value = 0.0
    else:
        value = float(value)
    return create_literal(value, datatype=XSD.float)


def create_int_literal(value):
    """
    Create an integer literal. 
    """
    return create_literal(value, datatype=XSD.integer)

Name normalization is handled by the `normalize_name` function, which replaces spaces with underscores and removes characters such as commas, periods, colons, and semicolons. This step guarantees that URIs generated for institutions and courses are syntactically valid and consistent, allowing for stable linking between entities in the RDF graph.

In [19]:
def normalize_name(value):
    """
    Normalize a name by replacing spaces with underscores and removing special characters.
    """
    return value.replace(" ", "_").replace(",", "").replace(".", "").replace("º", "").replace(":", "").replace(";", "")

To construct a meaningful and queryable representation of the Portuguese higher education system, a series of functions were defined to map key entities and their relationships into RDF triples. These entities include institutions, courses, degrees, scientific areas, and their respective geolocations. Each function plays a distinct role in enriching the semantic structure of the knowledge graph.

The `create_institution` function adds institutions to the graph by assigning them a URI based on a normalized version of their name. It associates each institution with its name and unique institutional code, ensuring that each one is distinct and queryable. Similarly, the `create_type_course` function defines courses with their name and code, while the create_type function allows the instantiation of unique course offerings by combining the institution and course names into a single URI. This disambiguation is crucial, especially in cases where the same course name may exist across different institutions.

In [20]:
def create_institution(g, name, code):
    """
    Create an institution in the graph with the given name and code.
    """
    inst = EDU[normalize_name(name)]
    literal = create_str_literal(name)
    add_to_graph(g, inst, RDF.type, EDU.Institution)
    add_to_graph(g, inst, EDU.institutionName, literal)
    add_to_graph(g, inst, EDU.institutionCode, create_int_literal(code))
    return inst


def create_type_course(g, name, code):
    """
    Create a course in the graph with the given name and code.
    """ 
    n = normalize_name(name)
    course = EDU[n]
    literal = create_str_literal(name)
    add_to_graph(g, course, RDF.type, EDU.Course)
    add_to_graph(g, course, EDU.courseName, literal)
    add_to_graph(g, course, EDU.courseCode, create_str_literal(code))
    return course


def create_type(g, course_type, inst_name, course_name):
    """
    Create a course type in the graph with the given name and code.
    """
    course = EDU[normalize_name(inst_name) + "_" + normalize_name(course_name)]
    add_to_graph(g, course, RDF.type, course_type)
    return course

To establish relationships between institutions and their offered courses, the `inst_has_course` function links them directly. The `create_degree` function introduces academic degrees, and the `course_degree` function connects these degrees to the courses that award them. Scientific classification is handled through the `create_scientific_area` function, which generates entities for scientific areas based on both name and numeric code. Each course is then associated with a relevant scientific area via the `course_scientific_area` function.

In [21]:
def inst_has_course(g, inst, course):
    """
    Associate an institution with a course in the graph.
    """
    add_to_graph(g, inst, EDU.hasCourse, course)


def create_degree(g, name):
    """
    Create a degree in the graph with the given name.
    """
    degree = EDU[normalize_name(name)]
    add_to_graph(g, degree, RDF.type, EDU.Degree)
    add_to_graph(g, degree, EDU.degreeName, create_str_literal(name))
    return degree


def course_degree(g, course, degree):
    """
    Associate a course with a degree in the graph.
    """
    add_to_graph(g, course, EDU.awardsDegree, degree)


def create_scientific_area(g, code, name):
    """
    Create a scientific area in the graph with the given code and name.
    """
    scientific_area = EDU[normalize_name(name)]
    add_to_graph(g, scientific_area, RDF.type, EDU.ScientificArea)
    add_to_graph(g, scientific_area, EDU.scientificAreaCode, create_int_literal(int(float(code))))
    add_to_graph(g, scientific_area, EDU.scientificAreaName, create_str_literal(name))
    return scientific_area


def course_scientific_area(g, course, scientific_area):
    """
    Associate a course with a scientific area in the graph.
    """
    add_to_graph(g, course, EDU.hasScientificArea, scientific_area)

The code also models key attributes that influence course selection. The `last_admitted_grade` function captures the minimum admission grade for each course, accommodating missing data by assigning a default of 0.0. Similarly, the `course_available_slots` function records the number of available slots, also defaulting to 0 when data is incomplete. This approach maintains data integrity and supports filtering in semantic queries.

In [22]:
def last_admitted_grade(g, course, grade):
    """
    Set the last admitted grade for a course in the graph.
    If the value is "---", it is converted to 0.0.
    """
    if grade == "---":
        grade = 0
    grade = create_float_literal(float(grade))
    add_to_graph(g, course, EDU.lastAdmittedGrade, grade)


def course_available_slots(g, course, slots):
    """
    Set the available slots for a course in the graph. 
    If the value is "---", it is converted to 0.
    """
    if slots == "---":
        slots = 0
    else:
        slots = int(float(slots))
    add_to_graph(g, course, EDU.availableSlots, create_int_literal(slots))

Geographical context is added through `create_district` and `create_county`, which define regional entities and their hierarchical relationship. The `add_location_to_institution` function integrates these spatial details into each institution's profile, including address and postal code. This geolocation data enables spatial querying and supports use cases such as regional policy planning or student access analysis.

In [23]:
def add_location_to_institution(g, inst, address, postal_code, district, county):
    """
    Add location information to an institution in the graph.
    """
    if address:
        add_to_graph(g, inst, EDU.address, create_str_literal(address))
    if postal_code:
        add_to_graph(g, inst, EDU.postalCode, create_str_literal(postal_code))
    if district:
        add_to_graph(g, inst, EDU.locatedInDistrict, district)
    if county:
        add_to_graph(g, inst, EDU.locatedInCounty, county)


def create_district(g, district_name):
    """
    Create a district in the graph with the given name.
    """
    district = EDU[normalize_name(district_name)]
    add_to_graph(g, district, RDF.type, EDU.District)
    add_to_graph(g, district, EDU.districtName, create_str_literal(district_name))
    return district


def create_county(g, county_name, district):
    """
    Create a county in the graph with the given name and associate it with a district.
    """
    county = EDU[normalize_name(county_name)]
    add_to_graph(g, county, RDF.type, EDU.County)
    add_to_graph(g, county, EDU.countyName, create_str_literal(county_name))
    add_to_graph(g, county, EDU.countyPartOfDistrict, district)
    return county


The central logic of graph population is encapsulated in the create_graph_from_excel function. This routine is responsible for orchestrating the complete transformation of structured tabular data into an RDF-based semantic model. It integrates information from two distinct Excel sources: one containing course and institutional data and another containing geolocation details.

Upon loading both datasets, the function initializes an RDF graph and binds it to a custom namespace for educational entities. A dictionary is then created to map institutional names to their corresponding location records, ensuring efficient lookup during the graph construction process.

The function iterates through each record in the main dataset, filtering out entries with missing institution or course names. For each valid entry, an institution node is created and enriched with a unique code. A course type is also instantiated, followed by the creation of a course entity that combines both the institution and course identifiers to ensure uniqueness.

The course is then linked to its institution, and additional attributes are incorporated as available. These include the number of available slots, the minimum admission grade from the previous year, the academic degree awarded, and the scientific area of study. Each attribute is modeled as a literal or linked entity, supporting both data-level and structural queries.

Geolocation data is integrated whenever available by cross-referencing the institution code. Districts and counties are instantiated or reused as necessary, and address information is attached directly to the institution entity. This enhances the spatial reasoning capabilities of the graph, enabling location-aware analyses.

Finally, the graph is serialized into Turtle format and saved to disk, either in the specified directory or in the default location. This serialized output represents a complete, machine-readable representation of the Portuguese higher education system, ready for semantic querying and reasoning.

In [55]:
def create_graph_from_excel(output_dir = None):
    """
    Create an RDF graph from the Excel file and save it as a Turtle file.
    """
    excel_file = "./docs/lista_de_vagas_para_1_fase.xlsx"
    location_file = "./docs/Localizacao.xlsx"

    df = pd.read_excel(excel_file, sheet_name=0, header=3)
    location_df = pd.read_excel(location_file)

    g = Graph()
    g.bind("edu", EDU)

    location_dict = {
        row["Código do Estabelecimento"]: row
        for _, row in location_df.iterrows()
    }

    for _, row in df.iterrows():
        if pd.isna(row["Nome da Instituição"]) or pd.isna(row["Nome do Curso"]):
            continue

        inst_name = get_value(row, "Nome da Instituição")
        course_name = get_value(row, "Nome do Curso")
        inst_code =int(float( get_value(row, "Código Instit.")))

        inst = create_institution(g, inst_name, inst_code)
        course_type = create_type_course(g, course_name, get_value(row, "Código Curso"))

        course = create_type(g, course_type, inst_name, course_name)

        inst_has_course(g, inst, course)

        available_slots = get_value(row, "Vagas 2024")
        if pd.notna(available_slots):
            course_available_slots(g, course, available_slots)

        degree = create_degree(g, get_value(row, "Grau"))
        course_degree(g, course_type, degree)
        
        scientific_area = create_scientific_area(g, get_value(row, "Área Científica"), get_value(row, 8))
        course_scientific_area(g, course_type, scientific_area)

        nota = get_value(row, "Nota último colocado 1ª Fase 2023 (cont. geral)")
        if pd.notna(nota):
            last_admitted_grade(g, course, nota)


        if inst_code in location_dict:
            location_data = location_dict[inst_code]
            district = create_district(g, location_data.get("Distrito"))
            county = create_county(g, location_data.get("Concelho"), district)
            add_location_to_institution(
                g,
                inst,
                location_data.get("Morada"),
                location_data.get("Código Postal"),
                district,
                county
            )

    file_name = "education.ttl"
    if output_dir:
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        file_name = os.path.join(output_dir, file_name)
    g.serialize(file_name, format="turtle")

create_graph_from_excel("ontology")
print("Successfully generated RDF with locations!")


C:\Users\joana\AppData\Local\Temp\ipykernel_20368\358696417.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return str(row[column]).strip()


Successfully generated RDF with locations!


### Queries

To enable semantic exploration of the data, the RDF graph previously serialized into Turtle format is reloaded into memory. This is achieved by instantiating a new RDF Graph object and parsing the Turtle file generated during the data transformation phase.

In [61]:
file = "ontology/education.ttl"
g = Graph()
g.parse(file, format="ttl")


<Graph identifier=Nd7631b408b354d9a919ee0221d9b418b (<class 'rdflib.graph.Graph'>)>

To retrieve targeted insights from the educational knowledge graph, SPARQL is used as the query language. In this example, a query is constructed to identify institutions that offer a course whose name includes the phrase Engenharia Informática, regardless of case sensitivity. The graph is queried for each matching institution's code, name, course name, and the grade of the last admitted student.

The query traverses relationships from institutions to their offered courses and associated metadata, leveraging the RDF and custom edu vocabulary. Although additional filtering options are present, such as restricting results to a specific district or grade threshold.

Once executed, the query returns a distinct list of institutions and their respective course details. 

In [66]:
query = """
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX edu: <http://example.org/education#>

  SELECT DISTINCT ?code ?name ?grade ?courseName
  WHERE {
    ?institution a edu:Institution;
                edu:institutionCode ?code ;
                edu:institutionName ?name ;
                # edu:locatedInDistrict ?location ;
                edu:hasCourse ?course .
    
    ?location edu:districtName ?districtName .

    ?course rdf:type ?courseType;
            edu:lastAdmittedGrade ?grade .
            
    ?courseType edu:courseName ?courseName ;
                edu:courseCode ?courseCode .
                
            
    FILTER(CONTAINS(LCASE(?courseName), LCASE("Engenharia Informática"^^xsd:string)))
    FILTER(LCASE(?districtName) = LCASE("Lisboa"^^xsd:string))
    FILTER(xsd:float(?grade) <= 150.0)
  }
"""

results = g.query(query)
print("List of universities:")
for row in results:
    print(f"{row.code} | {row.name} | {row.courseName} | {row.grade}")

List of universities:
7105 | Escola Superior Náutica Infante D. Henrique | Engenharia Informática e de Computadores | 133.9
6800 | ISCTE - Instituto Universitário de Lisboa | Engenharia Informática (regime pós-laboral) | 145.0
3092 | Instituto Politécnico da Guarda - Escola Superior de Tecnologia e Gestão | Engenharia Informática | 123.0
3023 | Instituto Politécnico de Beja - Escola Superior de Tecnologia e de Gestão | Engenharia Informática | 124.9
3043 | Instituto Politécnico de Bragança - Escola Superior de Tecnologia e de Gestão de Bragança | Engenharia Informática | 115.5
3053 | Instituto Politécnico de Castelo Branco - Escola Superior de Tecnologia de Castelo Branco | Engenharia Informática | 108.0
3065 | Instituto Politécnico de Coimbra - Escola Superior de Tecnologia e Gestão de Oliveira do Hospital | Engenharia Informática | 112.7
3064 | Instituto Politécnico de Coimbra - Instituto Superior de Engenharia de Coimbra | Engenharia Informática | 145.8
3064 | Instituto Politécnico 

This SPARQL query targets a specific educational institution to extract detailed course information filtered by scientific area and academic performance. It focuses on the institution _Escola Superior Náutica Infante D. Henrique_ and retrieves all courses where the scientific area is _Eletricidade e energia_ and the grade of the last admitted student does not exceed 150.

The query traverses multiple relationships: from the institution to its courses, and then to their types, awarded degrees, and scientific areas. The output includes the institution name, course name, last admitted grade, scientific area, and the corresponding degree.

This query structure illustrates the power of semantic data modeling, enabling fine-grained, multi-criteria filtering over interconnected educational data.

In [69]:
query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX edu: <http://example.org/education#>

    SELECT DISTINCT ?institutionName ?courseName ?grade ?scientificArea ?degree
    WHERE {
            ?institution a edu:Institution;
                edu:institutionName ?institutionName ;
                edu:hasCourse ?course .

            ?course edu:lastAdmittedGrade ?grade ;
            		rdf:type ?courseType .

            ?courseType edu:courseName ?courseName ;
            			edu:awardsDegree ?degree_type ;
                		edu:hasScientificArea ?sa .

            ?degree_type edu:degreeName ?degree .

            ?sa edu:scientificAreaName ?scientificArea .
            
        FILTER(?institutionName = "Escola Superior Náutica Infante D. Henrique"^^xsd:string)
        FILTER(xsd:float(?grade) <= 150)
        FILTER(?scientificArea = "Eletricidade e energia"^^xsd:string)
    }
"""

results = g.query(query)
print("List of universities:")
for row in results:
    print(f"{row.institutionName} | {row.courseName} | {row.grade} | {row.scientificArea} | {row.degree}")

List of universities:
Escola Superior Náutica Infante D. Henrique | Engenharia Eletrotécnica Marítima | 131.6 | Eletricidade e energia | Licenciatura - 1º ciclo


## App

How to run the app from a python script

To make the RDF knowledge graph accessible via a user-friendly interface, a web application was developed. This application is built using JavaScript, with an Express backend and a React frontend.

The RDF graph, previously generated and serialized in Turtle format, is now hosted and served through a GraphDB instance, accessible at the following [endpoint](http://kg-project.fly.dev/repositories/kg_education).

The backend Express server is responsible for handling HTTP requests from the React frontend, constructing and sending SPARQL queries to the GraphDB endpoint, and returning the parsed results in JSON format.

The frontend allows users to interact with the knowledge graph through a graphical interface. Users can perform SPARQL queries without needing to understand the underlying syntax. Currently, the application supports the following two queries. Course search by university and/or scientific area, with an optional filter by admission grade; and institution search based on the desired course, with filters for district and admission grade.

To run the application locally, it is necessary to have Node.js installed, along with the following dependencies:
* [express](https://www.npmjs.com/package/express)
* [cors](https://www.npmjs.com/package/cors)
* [axios](https://www.npmjs.com/package/axios)
* [npm-run-all](https://www.npmjs.com/package/npm-run-all)
* [react-scripts](https://www.npmjs.com/package/react-scripts)
* [web-vitals](https://www.npmjs.com/package/web-vitals)

These can be installed by running the following line:

In [4]:
import platform
import subprocess

is_windows = platform.system() == 'Windows'
subprocess.run(["npm", "install"], cwd="app", shell=is_windows) 

CompletedProcess(args=['npm', 'install'], returncode=0)

Once the dependencies are installed, the app can be started. This command launches both the backend and frontend servers concurrently, enabling real-time interaction with the knowledge graph. The frontend is available at this [link](http://localhost:3000/).

In [ ]:
import platform
import subprocess

is_windows = platform.system() == 'Windows'
subprocess.run(["npm", "start"], cwd="app", shell=is_windows) 